In [28]:
import pandas as pd
import holidays 
from datetime import time
from datetime import datetime
from datetime import timedelta
from datetime import date
from datetime import time

In [29]:
# From here you can write the location of your excel file and you are ready to go    
df = pd.read_excel('Sample.xlsx')

The algorithm to automate the paycheck amount (part 3)

In [30]:

class EmployeePaycheck:
    def __init__(self, regular_working_hours=9, weekly_working_days=5, check_period_week=2,regular_price=100,nightwork_price=125,holiday_price=130,overtime_price_50_minus=120,overtime_price_50_plus=115):
        self.regular_working_hours = regular_working_hours
        self.weekly_working_days = weekly_working_days
        self.check_period_week =  check_period_week
        self.regular_price = regular_price
        self.nightwork_price = nightwork_price
        self.holiday_price = holiday_price
        self.overtime_price_50_minus = overtime_price_50_minus
        self.overtime_price_50_plus = overtime_price_50_plus

    def preparation(self, df):
        # Converting Date column to datetime format
        df['Date'] = pd.to_datetime(df['Date'])
    
        # Function to convert time to datetime 
        def convert_time(row, column):
            time_str = row[column]
            if isinstance(time_str, str):
                try:
                    return pd.to_datetime(time_str, format='%I:%M %p').time()
                except ValueError:
                    pass
            return time_str

        # Applying the function to Time in and Time out columns
        df['Time In'] = df.apply(convert_time, axis=1, column='Time In')
        df['Time Out same day or next day'] = df.apply(convert_time, axis=1, column='Time Out same day or next day')

        # Function to adjust Time out based on meridian
        def adjust_timeout(row):
            if row['Time Out same day or next day'] < row['Time In']:
                return pd.Timestamp.combine(pd.Timestamp(row['Date']).date() + pd.DateOffset(days=1), row['Time Out same day or next day'])
            else:
                return pd.Timestamp.combine(pd.Timestamp(row['Date']).date(), row['Time Out same day or next day'])

        def merge(row): 
            return  pd.Timestamp.combine(pd.Timestamp(row['Date']).date(),row['Time In'])
    
        # Applying the function to adjust Time out
        df['Time Out same day or next day'] = df.apply(adjust_timeout, axis=1)
        df['Time In'] = df.apply(merge,axis=1)


    def calculate_extra_hours(self, row):
      # calculating the total working hours a day 

        time_in = row['Time In']
        time_out = row['Time Out same day or next day']

        working_hours = pd.Timedelta(hours=0)
        time_after = time_in
        overtime = pd.Timedelta(hours=0)
        holiday = pd.Timedelta(hours=0)
        nightwork = pd.Timedelta(hours=0)
    
    
        # The points where they have to Consider a break ----------------->
        table = [3.5,7,10.5,14,17.5,21,24.5,28,31.5,35,38.5,42,45.5,49]


        while time_after < time_out:
             for i in table:
                if time_after-time_in == pd.Timedelta(hours=i):
                   time_after = time_after + pd.Timedelta(hours=0.5)
                   break
         
             # if there is not a 30 minutes time gap between the time in and time out
             if time_after > time_out:
                  break

             if time_after +pd.Timedelta(hours=0.5) > time_out:
                  break
         
             # increasing the working hours by 30 minutes until it reaches 8 hours 9 hours and 2 30 minutes break
             if working_hours < pd.Timedelta(hours=8):
                 working_hours += pd.Timedelta(hours=0.5)
                 time_after +=pd.Timedelta(hours=0.5)

             # then we are shifting to the holidays is it a national holiday or not using holiday package
             # investigating the testcase where holiday ends before t+30 minutes
             elif time_after.date() in holidays.India(time_after.year) : 
                  timestamp12am = pd.Timestamp.combine(time_after.date()+pd.DateOffset(1), pd.Timestamp('00:00').time())
                  if time_after + pd.Timedelta(hours=0.5) > timestamp12am:
                      holiday += min(timestamp12am-time_after,pd.Timedelta(hours=0.5))
                      overtime  += max((pd.Timedelta(hours=0.5)-min(timestamp12am-time_after,pd.Timedelta(hours=0.5))),pd.Timedelta(hours=0))
                      time_after += pd.Timedelta(hours=0.5)
                  else : 
                      holiday += pd.Timedelta(hours=0.5)     
                      time_after = time_after + pd.Timedelta(hours=0.5)

            # after which we are prioritising the case where he is been working bw 2 to 5 and 
            # resolving the case where  t+30 minutes exceeds 5 am
             elif time_after.time() >= time(2,0) and time_after.time() <= time(5,0):
                date = time_after.date()
                timestamp5am = pd.Timestamp.combine(date, pd.Timestamp('05:00').time())
                nightwork += min((timestamp5am-time_after),pd.Timedelta(hours=0.5))
                overtime += max((pd.Timedelta(hours=0.5)-min((timestamp5am-time_after),pd.Timedelta(hours=0.5))),pd.Timedelta(hours=0))
                time_after = time_after + pd.Timedelta(hours=0.5)


            
            # if none of the above case is applied we consider it as an overtime and 
            # looking to resolve the boundary case at 12 am and 2 am
             else :
                timestamp12am = pd.Timestamp.combine(time_after.date() + pd.DateOffset(1), pd.Timestamp('00:00').time())
                timestamp2am = pd.Timestamp.combine(time_after.date(), pd.Timestamp('02:00').time())
                if time_after + pd.Timedelta(hours=0.5) > timestamp12am and timestamp12am.date() in holidays.India(timestamp12am.year) :
                      overtime += min((timestamp12am-time_after),pd.Timedelta(hours=0.5))
                      holiday += max((pd.Timedelta(hours=0.5)-min((timestamp12am-time_after),pd.Timedelta(hours=0.5))),pd.Timedelta(hours=0))
                      time_after = time_after + pd.Timedelta(hours=0.5)
                elif time_after + pd.Timedelta(hours=0.5) >= timestamp2am and time_after - pd.Timedelta(hours=0.5) <= timestamp2am:
                      overtime += min((timestamp2am-time_after),pd.Timedelta(hours=0.5))
                      nightwork += max((pd.Timedelta(hours=0.5)-min((timestamp2am-time_after),pd.Timedelta(hours=0.5))),pd.Timedelta(hours=0))
                      time_after = time_after + pd.Timedelta(hours=0.5)
                else:
                      overtime += pd.Timedelta(hours=0.5)
                      time_after = time_after + pd.Timedelta(hours=0.5)


    
        if working_hours < pd.Timedelta(hours=8):
            working_hours += max(time_out - time_after,pd.Timedelta(hours=0)) 
            time_after = time_out

        elif time_after.date() in holidays.India(time_after.year) :
            time_diff = time_out - time_after
            timestamp12am = pd.Timestamp.combine(time_after.date()+pd.DateOffset(1), pd.Timestamp('00:00').time())
            if time_after + time_diff > timestamp12am:
                      holiday += min(timestamp12am-time_after,time_diff)
                      overtime += max((time_diff-min(timestamp12am-time_after,time_diff)),pd.Timedelta(hours=0))
                      time_after += time_diff
            else: 
                holiday += max(time_out - time_after,pd.Timedelta(hours=0)) 
                time_after = time_out

        elif time_after.time() >= time(2,0) and time_after.time() <= time(5,0):
                date = time_after.date()
                time_diff = time_out - time_after
                timestamp5am = pd.Timestamp.combine(date, pd.Timestamp('05:00').time())
                nightwork += min((timestamp5am-time_after),time_diff)
                overtime += max((time_diff-min((timestamp5am-time_after),time_diff)),pd.Timedelta(hours=0))
                time_after = time_after + time_diff
        else:
                timestamp12am = pd.Timestamp.combine(time_after.date() + pd.DateOffset(1), pd.Timestamp('00:00').time())
                timestamp2am = pd.Timestamp.combine(time_after.date(), pd.Timestamp('02:00').time())
                time_diff = time_out - time_after

                if time_after + time_diff > timestamp12am and timestamp12am.date() in holidays.India(time_after.year) :
                    overtime += min((timestamp12am-time_after),time_diff)
                    holiday += max((time_diff-min((timestamp12am-time_after),time_diff)),pd.Timedelta(hours=0))
                    time_after = time_out

                elif time_after + time_diff >= timestamp2am and time_after - time_diff <= timestamp2am:
                    overtime += min((timestamp2am-time_after),time_diff)
                    nightwork += max((time_diff-min((timestamp2am-time_after),time_diff)),pd.Timedelta(hours=0))
                    time_after = time_out
                
                else:
                    overtime += (time_out - time_after)
                    time_after = time_out
    
    
        return working_hours,overtime, nightwork,holiday

    def paycheck(self, df):
        paycheck = []
        # iterating over the rows of dataframe
        for index, row in df.iterrows():
            a,b,c,d = 0,0,0,0
            regular = []
            paybill = 0
            cnt = 0
            if cnt+self.check_period_week*7 > len(df):
                break
            for i in range(cnt,cnt+self.check_period_week*7):
                if i < len(df):
                    working_hours,overtime,nightwork,holiday = self.calculate_extra_hours(df.iloc[i])
                    a += working_hours.total_seconds()/3600
                    b += overtime.total_seconds()/3600
                    c += nightwork.total_seconds()/3600
                    d += holiday.total_seconds()/3600
                    regular.append(working_hours.total_seconds()/3600)
            
            sorted_regular = sorted(regular)
            b+=(sum(sorted_regular[:self.check_period_week*(7-self.weekly_working_days)]))
            a-=(sum(sorted_regular[:self.check_period_week*(7-self.weekly_working_days)]))
            cnt+=self.check_period_week*7
            
            paybill+=a*self.regular_price
           
            if b+c+d>=50:
                paybill += b*self.overtime_price_50_plus+c*self.nightwork_price+d*self.holiday_price
            else : 
                paybill+= b*self.overtime_price_50_minus+c*self.nightwork_price+d*self.holiday_price
            
        paycheck.append(paybill)
        
        return paycheck

Evaluating the biweekly_payment_amount for the given scenario(part 2)

In [31]:
# Evaluating the paycheck of the employee as stated in the 2 part using all the given parameters as default parmeters
nurse = EmployeePaycheck()
nurse.preparation(df)
payment = nurse.paycheck(df)
print(payment)

# the answer is coming as 16324.5 ruppess

[16324.5]


Preparing the test cases and testing (part 1)


In [36]:
import pytest
import numpy as np
import pandas as pd

# Its always advicable to test your algrotihm with large set of random testcases
# This will help you to understand the flaws in your algorithm and also help you to improve it
dates = pd.date_range(start='2021-01-01', periods=1000)
time_in = np.random.choice(['08:00', '18:00', '08:30', '18:30'], size=1000)
time_out = np.random.choice(['17:00', '09:00', '17:30', '18:30'], size=1000)


testcase = pd.DataFrame({
    'Date': dates,
    'Time In': time_in,
    'Time Out same day or next day': time_out
})

testcase.sample(45)

,Date,Time In,Time Out same day or next day
687,2022-11-19,18:00,09:00
326,2021-11-23,18:30,17:00
674,2022-11-06,08:00,17:30
899,2023-06-19,18:30,17:30
61,2021-03-03,18:30,09:00
226,2021-08-15,18:30,18:30
719,2022-12-21,18:30,17:30
236,2021-08-25,08:30,09:00
7,2021-01-08,08:30,17:00
532,2022-06-17,08:30,17:00


In [37]:
# all the testcases passed indication that our algorithm is good to go 

def test_preparation():
    ep = EmployeePaycheck()
    prepared_df = ep.preparation(testcase)
    assert 'new_column' in prepared_df.columns

def test_calculate_extra_hours():
    ep = EmployeePaycheck()
    prepared_df = ep.preparation(testcase)
    row = prepared_df.iloc[0]
    working_hours, overtime, nightwork, holiday = ep.calculate_extra_hours(row)
    assert working_hours >= 0
    assert overtime >= 0
    assert nightwork >= 0
    assert holiday >= 0

def test_paycheck():
    ep = EmployeePaycheck()
    val = ep.paycheck(testcase)
    assert(val[:]>0)